In [ ]:
# Comment this when in local runtime

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install accelerate

In [ ]:
! pip install transformers[torch] datasets evaluate

# !pip install transformers[torch]

In [ ]:
import numpy as np
# # Drive Path
Xtrain = np.load('/content/drive/MyDrive/COLIEE data/data/Xtrain_For_H30.npy')
Ytrain = np.load('/content/drive/MyDrive/COLIEE data/data/Ytrain_For_H30.npy')

# # Local Path  D:\ResearchPhD\COLIEE\data\data
# Xtrain = np.load('/mnt/d/ResearchPhD/COLIEE/data/data/Xtrain_For_H30.npy')
# Ytrain = np.load('/mnt/d/ResearchPhD/COLIEE/data/data/Ytrain_For_H30.npy')

In [ ]:
from datasets import Dataset
# d={}
# d["train"]={}

# d["train"]["text"]=[]
# d["train"]["labels"]=[]
# for i in range(0,len(X_train)):
#   d["train"]["text"].append("<sent-1> "+str(X_train[i][0], encoding='utf-8')+"<sent-1>"+" <sent-2> "+str(X_train[i][1], encoding='utf-8')+"<sent-2>")
#   d["train"]["labels"].append(y_train[i])



# d["test"]={}

# d["test"]["text"]=[]
# d["test"]["labels"]=[]
# for i in range(0,len(X_val)):
#   d["test"]["text"].append("<sent-1> "+str(X_val[i][0], encoding='utf-8')+"<sent-1>"+" <sent-2> "+str(X_val[i][1], encoding='utf-8')+"<sent-2>")
#   d["train"]["labels"].append(y_val[i])


# ds = Dataset.from_dict(d)


d={}

d["text1"]=[]
d["text2"]=[]
d["label"]=[]
for i in range(0,len(Xtrain)):
  d["text1"].append(str(Xtrain[i][0], encoding='utf-8').lower())
  d["text2"].append(str(Xtrain[i][1], encoding='utf-8').lower())
  d["label"].append(0 if Ytrain[i]=='N' else 1)

ds = Dataset.from_dict(d)

# d={}

# d["text"]=[]
# d["labels"]=[]
# for i in range(0,len(X_val)):
#   d["text"].append("<sent-1> "+str(X_val[i][0], encoding='utf-8')+"<sent-1>"+" <sent-2> "+str(X_val[i][1], encoding='utf-8')+"<sent-2>")
#   d["labels"].append(y_val[i])

# ds2 = Dataset.from_dict(d)

In [ ]:
train_test = ds.train_test_split(test_size=0.1)
# train_test


In [ ]:
train_test

DatasetDict({
    train: Dataset({
        features: ['text1', 'text2', 'label'],
        num_rows: 896
    })
    test: Dataset({
        features: ['text1', 'text2', 'label'],
        num_rows: 100
    })
})

In [ ]:
train_test = ds.train_test_split(test_size=0.1)

In [ ]:
testing_set = train_test['test']

In [ ]:
training_set= train_test['train']



In [ ]:
# minillm download --model llama-7b-4bit --weights llama-7b-4bit.pt

In [ ]:
# import transformers
# import torch
# tokenizer = transformers.LlamaTokenizer.from_pretrained('chaoyi-wu/PMC_LLAMA_7B')
# model = transformers.LlamaForCausalLM.from_pretrained('chaoyi-wu/PMC_LLAMA_7B')
# sentence = 'Hello, doctor'
# batch = tokenizer(
#             sentence,
#             return_tensors="pt",
#             add_special_tokens=False
#         )

In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

## v2 models
model_path = 'openlm-research/open_llama_3b_v2'
# model_path = 'openlm-research/open_llama_7b_v2'

## v1 models
# model_path = 'openlm-research/open_llama_3b'
# model_path = 'openlm-research/open_llama_7b'
# model_path = 'openlm-research/open_llama_13b'

tokenizer = LlamaTokenizer.from_pretrained(model_path)
# model = LlamaForCausalLM.from_pretrained(
#     model_path, torch_dtype=torch.float16, device_map='cuda'
# )

# prompt = 'Q: What is the largest animal?\nA:'
# input_ids = tokenizer(prompt, return_tensors="pt", device_map='cuda').input_ids.to('cuda')

# generation_output = model.generate(
#     input_ids=input_ids, max_new_tokens=32
# )
# print(tokenizer.decode(generation_output[0]))

In [ ]:
from transformers import AutoModel
import torch.nn as nn
import torch.optim as optim
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class LEGAL_BERT(nn.Module):
    def __init__(self):
        super(LEGAL_BERT, self).__init__()
        # self.bert_model_sent1 = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased").to(device)
        self.bert_model_sent1 =LlamaForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map='cuda').to(device)
        self.bert_model_sent1.requires_grad=False
        # self.bert_model_sent2 = AutoModel.from_pretrained("nlpaueb/bert-base-uncased-echr").to(device)
        # self.bert_model_sent2.requires_grad=False
        self.out = nn.Linear(32000, 1)

    def forward(self,ids,mask,token_type_ids1):
        # _,s1= self.bert_model_sent1(ids,attention_mask=mask, token_type_ids=token_type_ids1,return_dict=False)
        _,s1= self.bert_model_sent1(ids,attention_mask=mask,return_dict=False)
        print(s1.shape)
        # _,s2= self.bert_model_sent2(ids2,attention_mask=mask2,token_type_ids=token_type_ids2, return_dict=False)
        out= self.out(s1)

        return out

model=LEGAL_BERT()



loss_fn = nn.BCEWithLogitsLoss()

#Initialize Optimizer
optimizer= optim.Adam(model.parameters(),lr= 1e-5)

In [ ]:
type(output)

In [ ]:
model.create_model_card

In [ ]:
# # import transformers
# from  transformers import AutoTokenizer,AutoModel
# import torch
# # from transformers import LlamaTokenizer, LlamaForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-uncased-echr")
# # tokenizer = transformers.LlamaTokenizer.from_pretrained('chaoyi-wu/PMC_LLAMA_7B')
# # tokenizer = transformers.LlamaTokenizer.from_pretrained('chaoyi-wu/PMC_LLAMA_7B')

# # model_path='openlm-research/open_llama_3b_v2'

# # tokenizer = LlamaTokenizer.from_pretrained(model_path)
# # model = LlamaForCausalLM.from_pretrained(
# #     model_path, torch_dtype=torch.float16, device_map='auto',
# # )
# # if tokenizer.pad_token is None:
# #     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# #     model.resize_token_embeddings(len(tokenizer))


In [ ]:
model

In [ ]:
import matplotlib.pyplot as plt
epochs = 10
BATCH_SIZE = 4
model.to(device)
from tqdm import tqdm
loss_history = []
prev_loss=5000
patience=5

model.train()

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

for  epoch in range(epochs):
  loss_agg = 0.0



  for index in range(int(len(training_set)/BATCH_SIZE)):
    x_batch = training_set[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
    sent1 = x_batch['text1']
    print(x_batch['text1'])
    print(x_batch['text1'][0])
    print(x_batch['text2'][0])
    print([x_batch['text1'][0]+x_batch['text2'][0]])
    # +x_batch['text2']
    # sent2 = x_batch['text2']
    label = x_batch['label']
    # ids1,token_type_ids1, mask1, ids2,token_type_ids2, mask2 = [],[],[], [],[],[]
    ids1,token_type_ids1, mask1 = [],[],[]
    for i in range(len(sent1)):
        inputs1 = tokenizer.encode_plus(
            sent1[i] ,
            None,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            max_length=512,
            truncation=True,
        )

        # input_ids = tokenizer(prompt, return_tensors="pt")
        ids1.append(inputs1["input_ids"])
        token_type_ids1.append(inputs1["token_type_ids"])
        mask1.append(inputs1["attention_mask"]    )

    #     def preprocess_function(examples):
    # return tokenizer(examples["sentence"], padding='max_length', truncation=True, return_token_type_ids=False)
        # inputs2 = tokenizer.encode_plus(
        #     sent2[i] ,
        #     None,
        #     pad_to_max_length=True,
        #     return_attention_mask=True,
        #     max_length=512,
        #     truncation=True,
        # )
        # ids2.append(inputs2["input_ids"])
        # token_type_ids2.append(inputs2["token_type_ids"])
        # mask2.append(inputs2["attention_mask"]    )
    ids1 = torch.tensor(ids1, dtype=torch.long).to(device)
    token_type_ids1 = torch.tensor(token_type_ids1, dtype=torch.long).to(device)
    mask1 = torch.tensor(mask1, dtype=torch.long).to(device)
    # ids2 = torch.tensor(ids2, dtype=torch.long).to(device)

    # token_type_ids2 = torch.tensor(token_type_ids2, dtype=torch.long).to(device)
    # mask2 = torch.tensor(mask2, dtype=torch.long).to(device)
    label = torch.tensor(label, dtype=torch.float32).to(device)
    label = label.unsqueeze(1)
    out = model(ids1,mask1,token_type_ids1).to(device)
    print(out)
    print(label)
    loss=loss_fn(out,label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pred = np.where(out.cpu() >= 0., 1, 0)
    num_correct = sum(1 for a, b in zip(pred, label) if a[0] == b[0])
    num_samples = pred.shape[0]
    accuracy = num_correct/num_samples
    loss_agg += loss.item()

    #TODO
    # Write the validation set accuracy
    # Aggregate the loss over all batches
    # Track both validation loss and training loss
    #How to combine the representations from both sentences
    # Save the best model
    print(f'Epoch number" {epoch} Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} | loss {loss}')

  avg_loss=loss_agg/4
  if (avg_loss)>prev_loss:
    patience-=1
    print("patience:"+str(patience))
  prev_loss=avg_loss

  loss_history.append(avg_loss)

  plt.plot(loss_history)

In [ ]:
#Test on Validation Set
import matplotlib.pyplot as plt
# epochs = 10
BATCH_SIZE = 4
# model.to(device)
from tqdm import tqdm

# for  epoch in range(epochs):
#   loss_agg = 0.0
#   loss_history = []
#   model.train()
model.eval()
preds=[]

for index in range(int(len(testing_set)/BATCH_SIZE)):
  print(index)
  x_batch = testing_set[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
  sent1 = x_batch['text1']
  sent2 = x_batch['text2']
  label = x_batch['label']
  ids1,token_type_ids1, mask1, ids2,token_type_ids2, mask2 = [],[],[], [],[],[]
  for i in range(len(sent1)):
      inputs1 = tokenizer.encode_plus(
            sent1[i] ,
            None,
            pad_to_max_length=True,
            return_attention_mask=True,
            max_length=512,
            truncation=True,
        )
      ids1.append(inputs1["input_ids"])
      token_type_ids1.append(inputs1["token_type_ids"])
      mask1.append(inputs1["attention_mask"]    )
      inputs2 = tokenizer.encode_plus(
            sent2[i] ,
            None,
            pad_to_max_length=True,
            return_attention_mask=True,
            max_length=512,
            truncation=True,
        )
      ids2.append(inputs2["input_ids"])
      token_type_ids2.append(inputs2["token_type_ids"])
      mask2.append(inputs2["attention_mask"]    )
  ids1 = torch.tensor(ids1, dtype=torch.long).to(device)
  token_type_ids1 = torch.tensor(token_type_ids1, dtype=torch.long).to(device)
  mask1 = torch.tensor(mask1, dtype=torch.long).to(device)
  ids2 = torch.tensor(ids2, dtype=torch.long).to(device)

  token_type_ids2 = torch.tensor(token_type_ids2, dtype=torch.long).to(device)
  mask2 = torch.tensor(mask2, dtype=torch.long).to(device)
  label = torch.tensor(label, dtype=torch.float32).to(device)
  label = label.unsqueeze(1)
  out = model(ids1,mask1,token_type_ids1, ids2,mask2,token_type_ids2).to(device)
  # loss=loss_fn(out,label)
  # optimizer.zero_grad()
  # loss.backward()
  # optimizer.step()
  pred = np.where(out.cpu() >= 0., 1, 0)
  preds.append(pred)
    # num_correct = sum(1 for a, b in zip(pred, label) if a[0] == b[0])
    # num_samples = pred.shape[0]
    # accuracy = num_correct/num_samples
    # loss_agg += loss.item()
    # loss_history.append(loss.item())
    #TODO
    # Write the validation set accuracy
    # Aggregate the loss over all batches
    # Track both validation loss and training loss
    #How to combine the representations from both sentences
    # Save the best model
  #   print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} | loss {loss}')

  # plt.plot(loss_history)

preds2=[i.squeeze() for i in preds]
preds2=np.concatenate(preds2).ravel().tolist()
len(preds2)

from sklearn.metrics import classification_report
print(classification_report(preds2, testing_set["label"]))

In [ ]:
#This is true test set
import xml.etree.ElementTree as ET

xval = []
yval = []

# for fl in ['testset/riteval_H30_en.xml']:  "G:\ResearchPhD\Mi Young\COLIEE work\COLIEE data\Samples\testset\TestData_en.xml"
for fl in ['/content/drive/MyDrive/COLIEE data/Samples/testset/TestData_en.xml']:
    tree = ET.parse(fl)
    root = tree.getroot()
    for item in root.findall('pair'):
        # yval.append(item.attrib['label'])
        yval.append(item.attrib['id'])
        sample = []
        for vals in item:
            if vals.tag == 't1':
                sample.append(vals.text.encode('utf8').lower().strip())
            elif vals.tag == 't2':
                sample.append(vals.text.encode('utf8').lower().strip())
        xval.append((sample[0], sample[1]))

yval=[]
file1 = open('/content/drive/MyDrive/COLIEE data/data/answer-task4', 'r')
count = 0

while True:
    line = file1.readline()
    if line== '':
        break
    line=line.split()[1]
    yval.append(line)

file1.close()

In [ ]:
len(yval)

In [ ]:
from datasets import Dataset
d={}

d["text1"]=[]
d["text2"]=[]
d["label"]=[]
for i in range(0,len(xval)):
  d["text1"].append(str(xval[i][0], encoding='utf-8').lower())
  d["text2"].append(str(xval[i][1], encoding='utf-8').lower())
  d["label"].append(0 if yval[i]=='N' else 1)

finaltestset = Dataset.from_dict(d)

In [ ]:
len(finaltestset)

In [ ]:
BATCH_SIZE = 4
x_batch = finaltestset[index*BATCH_SIZE:(index+1)*BATCH_SIZE]

In [ ]:
len(x_batch)

In [ ]:
#Test on Test Set

import matplotlib.pyplot as plt
# epochs = 10
BATCH_SIZE = 4
# model.to(device)
from tqdm import tqdm

# for  epoch in range(epochs):
#   loss_agg = 0.0
#   loss_history = []
#   model.train()
model.eval()
preds=[]
count=0
for index in range(int(len(finaltestset)/BATCH_SIZE)+1):
  print(index)
  count+=1
  if count==26:
    x_batch=finaltestset[100:101]
  else:
    x_batch = finaltestset[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
  print(index*BATCH_SIZE,(index+1)*BATCH_SIZE,count)
  sent1 = x_batch['text1']
  sent2 = x_batch['text2']
  label = x_batch['label']
  ids1,token_type_ids1, mask1, ids2,token_type_ids2, mask2 = [],[],[], [],[],[]
  for i in range(len(sent1)):
      inputs1 = tokenizer.encode_plus(
            sent1[i] ,
            None,
            pad_to_max_length=True,
            return_attention_mask=True,
            max_length=512,
            truncation=True,
        )
      ids1.append(inputs1["input_ids"])
      token_type_ids1.append(inputs1["token_type_ids"])
      mask1.append(inputs1["attention_mask"]    )
      inputs2 = tokenizer.encode_plus(
            sent2[i] ,
            None,
            pad_to_max_length=True,
            return_attention_mask=True,
            max_length=512,
            truncation=True,
        )
      ids2.append(inputs2["input_ids"])
      token_type_ids2.append(inputs2["token_type_ids"])
      mask2.append(inputs2["attention_mask"]    )
  ids1 = torch.tensor(ids1, dtype=torch.long).to(device)
  token_type_ids1 = torch.tensor(token_type_ids1, dtype=torch.long).to(device)
  mask1 = torch.tensor(mask1, dtype=torch.long).to(device)
  ids2 = torch.tensor(ids2, dtype=torch.long).to(device)

  token_type_ids2 = torch.tensor(token_type_ids2, dtype=torch.long).to(device)
  mask2 = torch.tensor(mask2, dtype=torch.long).to(device)
  label = torch.tensor(label, dtype=torch.float32).to(device)
  label = label.unsqueeze(1)
  out = model(ids1,mask1,token_type_ids1, ids2,mask2,token_type_ids2).to(device)
  # loss=loss_fn(out,label)
  # optimizer.zero_grad()
  # loss.backward()
  # optimizer.step()
  pred = np.where(out.cpu() >= 0., 1, 0)
  preds.append(pred)

# preds2=[i.squeeze() for i in preds]
# preds2=np.concatenate(preds2).ravel().tolist()
# len(preds2)

preds2 = []
for sub_list in preds:
    for item in sub_list:
        preds2.append(item)

preds3=np.concatenate(preds2).ravel().tolist()
preds3, len(preds3)

from sklearn.metrics import classification_report
print(classification_report(np.concatenate(preds2).tolist(), finaltestset["label"]))

In [ ]:
preds3=np.concatenate(preds2).ravel().tolist()
print(preds3, len(preds3))

from sklearn.metrics import classification_report
print(classification_report(preds3, finaltestset["label"]))

In [ ]:
len(preds3)

In [ ]:
import pickle
with open('/content/drive/MyDrive/COLIEE samples/legalBERT_echr_predictions.pkl', 'wb') as f:  # open a text file
    pickle.dump(preds3, f) # serialize the list

In [ ]:
torch.save(model, '/content/drive/MyDrive/COLIEE samples/model_scripted.pt')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')